In [ ]:
from datetime import datetime
import time
import pandas as pd
import pickle
from ib_insync import *
import nest_asyncio
import logging,sys
from k_producer import *
from util_redis import *



def histBar(contract_d,param_d,useRTH=False):
    try:
        print('starting')
        df = pd.DataFrame()
        contract = Stock(contract_d['symbol'],contract_d['exchange'], contract_d['currency'])
        bars = ib.reqHistoricalData(contract,endDateTime=param_d['endt'], durationStr=param_d['dur'],barSizeSetting=param_d['candles'], whatToShow=param_d['show'], useRTH=useRTH)
        df = util.df(bars)
        
        df = df.set_index('date')
#         df.index.tz_localize('America/Toronto', nonexistent='shift_forward')
        df.columns = map(str.lower, df.columns)
        display(df.reset_index().dtypes)
        display(df)
    except Exception as e: print(e)
    finally:
        print('ended')
        return df
    
if __name__ == '__main__':
    logging.getLogger("asyncio").setLevel(logging.INFO)
    logging.basicConfig(
    level=logging.INFO,
    format='%(threadName)10s %(name)18s: %(message)s',stream=sys.stderr,)

    nest_asyncio.apply()
    
    util.startLoop()
    ib = IB()
    ib.connect('host.docker.internal', 7497, clientId=15)
    

    try:
        while True:
            time.sleep(1)
            item = lpop('hist.queue')
            if item !=None: 
                data = histBar(item['contract'],item['params'])
                if len(data)!=0:
                    p = pickle.dumps(data)
                    with open('t1d.pickle', 'wb') as f:
                        pickle.dump(data,f)
#                     itemr = {**item,**{'pdata':p}}
#                     print(itemr)
                    ProduceB('tester',item,p)
    except Exception as e: 
        print("FATAL: ",e)
    finally:ib.disconnect()

MainThread   ib_insync.client: Connecting to host.docker.internal:7497 with clientId 15...
MainThread   ib_insync.client: Connected
MainThread   ib_insync.client: Logged on to server version 152
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfuture.nj
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfuture
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfarm
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:ushmds.nj
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:ushmds
MainThread  ib_insync.wrapper: Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
MainThread   ib_insync.client: API connection ready
MainThread       ib_insync.ib: Synchronization complete


starting


date        datetime64[ns]
open               float64
high               float64
low                float64
close              float64
volume               int64
average            float64
barcount             int64
dtype: object

,open,high,low,close,volume,average,barcount
date,,,,,,,
2020-10-23 04:04:30,345.42,345.42,345.42,345.42,1,345.420,1
2020-10-23 04:04:45,345.65,345.65,345.65,345.65,10,345.650,1
2020-10-23 04:05:00,345.80,345.83,345.80,345.83,3,345.810,2
2020-10-23 04:05:15,345.83,345.83,345.83,345.83,0,345.830,0
2020-10-23 04:05:30,345.76,345.76,345.76,345.76,7,345.760,3
...,...,...,...,...,...,...,...
2020-10-23 19:58:45,345.49,345.49,345.49,345.49,3,345.490,1
2020-10-23 19:59:00,345.50,345.50,345.49,345.49,15,345.493,4
2020-10-23 19:59:15,345.50,345.50,345.50,345.50,2,345.500,1


ended
